In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu
from DataLoader import DataLoader
from halfkp import get_halfkp_indeicies
import chess
from chess import Board

import numpy as np

2022-03-29 11:27:20.926431: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-29 11:27:20.926458: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
training_generator = DataLoader(128, "train")
val_generator = DataLoader(128, "val")

In [6]:
from keras import backend as K
def clipped_relu(x):
    return K.relu(x, max_value=4)

In [9]:
halfAcc = Dense(256, clipped_relu)
i1 = tf.keras.Input((64*64*12,))
i2 = tf.keras.Input((64*64*12,))
acc1 = halfAcc(i1)
acc2 = halfAcc(i2)
layerStack = tf.keras.layers.Concatenate()([acc1, acc2])

h1 = Dense(32, clipped_relu)
h2 = Dense(32, clipped_relu)
out = Dense(1, 'sigmoid')

layerStack = out(h2(h1(layerStack)))

model = tf.keras.Model(inputs=[i1, i2], outputs=layerStack)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse', metrics=['mae'])

In [8]:
model = tf.keras.models.load_model("production/")

In [10]:
#2890/2890 [==============================] - 109s 38ms/step - loss: 0.2221 - mae: 0.4492 - val_loss: 0.2205 - val_mae: 0.4457
#2890/2890 [==============================] - 100s 35ms/step - loss: 0.2091 - mae: 0.4277 - val_loss: 0.2259 - val_mae: 0.4423
model.fit(training_generator, validation_data=val_generator, epochs=1)

312/312 [==============================] - 76s 242ms/step - loss: 0.0733 - mae: 0.2014 - val_loss: 0.1003 - val_mae: 0.2365


In [11]:
indicies = get_halfkp_indeicies(Board("rnb2bnr/pp1kpppp/3p4/2p5/4P3/1P6/P1PP1PPP/RNBQK1NR w KQ - 0 5"))
indicies = indicies.reshape(2, -1)
indicies = (np.array([indicies[0], indicies[0]]), np.array([indicies[1], indicies[1]]))
pred = model.predict(indicies)
print(pred[0])

[0.7342551]


In [18]:
pred = model.predict(val_generator[0][0]) * 2 - 1
act = val_generator[0][1] * 2 - 1
for i in range(128):
    print(pred[i], act[i], val_generator.getfen(0)[i])

[-0.04949099] 0.02425539296249868 r1bq1rk1/1ppp1ppp/p1n2n2/1Bb1p3/4P3/P1N2N1P/1PPP1PP1/R1BQK2R w KQ - 0 7
[-0.02719867] -0.6109751454932664 3r1rk1/4bpp1/pq2b2p/3pP3/4NB2/1P1Q4/P1P2PPP/3R1RK1 b - - 0 20
[0.35908425] -0.020791418713248633 r2q2k1/pppb2pn/3p3p/3P4/5B2/2N3P1/PPP2Q1P/R5K1 b - - 3 22
[0.0639931] -0.017326945218914314 rnbqkbnr/pp2pppp/2pp4/8/4P3/5Q2/PPPP1PPP/RNB1KBNR w KQkq - 0 3
[-0.1036787] -0.31472309449890357 r1bq1rk1/b5pp/p4p2/1p3P2/1P1pNR2/P6P/2P1B1P1/1R1Q3K b - - 0 20
[-0.22453517] -0.7504007867771375 2kr3r/ppp1qp2/4pb1p/6p1/3P3P/2P3P1/PP2Q1bR/RN2K3 w Q - 0 17
[0.02252209] -0.12753395844224835 r2q1rk1/ppp1bppn/3p3p/3Pn3/4P1b1/2N3P1/PPP1N1BP/R1B1QRK1 w - - 3 13
[-0.14433706] -0.7348611154986888 6k1/5pp1/7p/p3P3/8/1P1bP3/P4KPP/8 w - - 0 30
[-0.1788454] -0.35168739210477495 r1b2rk1/b5pp/p4p2/1p3P2/1PqpNR2/P6P/2P1B1P1/1R1Q3K b - - 4 22
[0.18866515] -0.7884973059452912 r2qkb1r/1ppb1p1p/p1np1np1/4p3/B3P3/1N1PBN2/PPP2PPP/R2QK2R b KQkq - 1 8
[0.2273469] 0.762264322827523 3nrr2/

In [7]:
import time
# model.save("models/model-{}".format(int(time.time())))
model.save("production/")

2022-03-29 11:22:07.600237: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: production/assets
